# Data preprocessing for credit card transaction data with Pandas

this notebook is describing how to preprocess data by using pandas.
Main concept is, 

<li> normalize dataset </li>
<li> split data to multiple chunks (training,validate,testing)</li>
<li> shuffle data by using ".sample(frac=1)" command</li>


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

df_csv = pd.read_csv('./data/creditcard.csv')
df_csv.head()

/Users/terrycho/anaconda/envs/tensorflow1.0/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
df_norm = (df_csv - df_csv.min() ) / (df_csv.max() - df_csv.min() )
df_norm.head()


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.000000,0.935192,0.766490,0.881365,0.313023,0.763439,0.267669,0.266815,0.786444,0.475312,...,0.561184,0.522992,0.663793,0.391253,0.585122,0.394557,0.418976,0.312697,0.005824,0.0
1,0.000000,0.978542,0.770067,0.840298,0.271796,0.766120,0.262192,0.264875,0.786298,0.453981,...,0.557840,0.480237,0.666938,0.336440,0.587290,0.446013,0.416345,0.313423,0.000105,0.0
2,0.000006,0.935217,0.753118,0.868141,0.268766,0.762329,0.281122,0.270177,0.788042,0.410603,...,0.565477,0.546030,0.678939,0.289354,0.559515,0.402727,0.415489,0.311911,0.014739,0.0
3,0.000006,0.941878,0.765304,0.868484,0.213661,0.765647,0.275559,0.266803,0.789434,0.414999,...,0.559734,0.510277,0.662607,0.223826,0.614245,0.389197,0.417669,0.314371,0.004807,0.0
4,0.000012,0.938617,0.776520,0.864251,0.269796,0.762975,0.263984,0.268968,0.782484,0.490950,...,0.561327,0.547271,0.663392,0.401270,0.566343,0.507497,0.420561,0.317490,0.002724,0.0


In [6]:
NORM_FILE='./data/creditcard_norm.csv'
df_norm.to_csv(NORM_FILE, index=False, header=False)

In [32]:
# split normalized data by label
df_norm_fraud=df_norm[ df_norm.Class==1.0] #fraud
df_norm_nonfraud=df_norm[ df_norm.Class==0.0] #non_fraud

# split non_fraudfor 60%,20%,20% (training,validation,test)
df_norm_nonfraud_train,df_norm_nonfraud_validate,df_norm_nonfraud_test = \
    np.split(df_norm_nonfraud,[int(.6*len(df_norm_nonfraud)),int(.8*len(df_norm_nonfraud))])
# split fraud data to 50%,50% (validation and test)
df_norm_fraud_validate,df_norm_fraud_test = \
    np.split(df_norm_fraud,[int(0.5*len(df_norm_fraud))])
print('number of non fraud training, test, validation dataset = ',\
len(df_norm_nonfraud_train),\
len(df_norm_nonfraud_test),\
len(df_norm_nonfraud_validate))
      
print('number of fraud test,fraud validation dataset =',\
len(df_norm_fraud_test),\
len(df_norm_fraud_validate))

#create train,validate and test dataset with shuffle
df_train = df_norm_nonfraud_train.sample(frac=1) 
df_validate = df_norm_nonfraud_validate.append(df_norm_fraud_validate).sample(frac=1)
df_test = df_norm_nonfraud_test.append(df_norm_fraud_test).sample(frac=1)

print 'size of train,validate,test data =',len(df_train),len(df_validate),len(df_test)

('number of non fraud training, test, validation dataset = ', 170589, 56863, 56863)
('number of fraud test,fraud validation dataset =', 246, 246)
size of train,validate,test data = 170589 57109 57109


# Training data set

In [33]:
df_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
93298,0.372367,0.954419,0.776836,0.851796,0.250897,0.761362,0.253067,0.265762,0.788579,0.448151,...,0.561689,0.505440,0.666979,0.440562,0.542865,0.573201,0.413948,0.312445,0.000260,0.0
97904,0.384462,0.951870,0.777518,0.860611,0.245702,0.766344,0.256660,0.269241,0.785535,0.454797,...,0.557118,0.474739,0.664348,0.361027,0.569121,0.442599,0.421292,0.315439,0.000116,0.0
45612,0.245509,0.978722,0.764601,0.855698,0.264455,0.760345,0.266597,0.259710,0.788292,0.482214,...,0.565002,0.540043,0.663224,0.322378,0.592799,0.525291,0.416654,0.313390,0.000475,0.0
140427,0.484490,0.978169,0.769782,0.842487,0.275388,0.764742,0.260190,0.264695,0.786396,0.455950,...,0.558081,0.481049,0.667638,0.378446,0.583870,0.443014,0.416292,0.313464,0.000077,0.0
14833,0.151130,0.951733,0.771451,0.854183,0.194120,0.769565,0.254323,0.269072,0.784140,0.457653,...,0.560693,0.497386,0.663796,0.327571,0.570267,0.556061,0.415246,0.313601,0.000550,0.0


# Test data set

In [34]:
df_test.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
246307,0.886169,0.992811,0.768657,0.811406,0.275558,0.766791,0.251983,0.266266,0.783060,0.480953,...,0.556140,0.469682,0.671604,0.514277,0.557919,0.449457,0.415372,0.312576,0.000035,0.0
274423,0.960698,0.963225,0.773564,0.829198,0.224029,0.774307,0.256663,0.275446,0.779682,0.462443,...,0.561827,0.524556,0.663150,0.449922,0.590048,0.369445,0.413245,0.308492,0.001946,0.0
241763,0.874867,0.988685,0.765713,0.805618,0.307440,0.770117,0.259261,0.269284,0.783008,0.458382,...,0.564143,0.528762,0.664359,0.479533,0.601135,0.334783,0.415742,0.312195,0.004476,0.0
238850,0.867233,0.992989,0.767583,0.793317,0.238376,0.768963,0.242995,0.270618,0.778120,0.474611,...,0.562490,0.526101,0.663937,0.393417,0.596027,0.599570,0.414123,0.312032,0.002225,0.0
239601,0.869126,0.940853,0.783305,0.812509,0.179597,0.776784,0.255228,0.275157,0.781137,0.479975,...,0.562438,0.544422,0.661823,0.396496,0.573278,0.435035,0.426159,0.319026,0.000057,0.0


# Validate data set

In [35]:
df_validate.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
195374,0.758253,0.994391,0.766892,0.795402,0.240267,0.771617,0.255203,0.268645,0.782723,0.468523,...,0.563952,0.531641,0.664680,0.414778,0.601929,0.462180,0.414650,0.311292,0.000090,0.0
184052,0.729617,0.998909,0.755032,0.806005,0.165802,0.762770,0.257086,0.261106,0.782855,0.399181,...,0.555960,0.481827,0.667883,0.226104,0.573490,0.382537,0.415967,0.311479,0.000224,0.0
177910,0.714142,0.987828,0.752844,0.818408,0.237912,0.760826,0.260679,0.263374,0.782805,0.448272,...,0.557820,0.488188,0.667215,0.470087,0.556403,0.508410,0.415325,0.313005,0.008682,0.0
224208,0.831613,0.937252,0.761968,0.845690,0.187771,0.754342,0.264694,0.259756,0.796445,0.455941,...,0.562671,0.502401,0.666398,0.477598,0.570658,0.359803,0.411319,0.307791,0.005316,0.0
226215,0.836480,0.993273,0.768703,0.808482,0.271187,0.768294,0.254425,0.266328,0.783182,0.480566,...,0.555761,0.467103,0.670786,0.459532,0.561483,0.453478,0.415390,0.312506,0.000077,0.0


# Write into files
write training, testing and validation data into files


In [38]:
TRAIN_FILE='creditcard_training.csv'
TEST_FILE='creditcard_tesring.csv'
VALIDATE_FILE='creditcard_validation.csv'

df_train.to_csv(TRAIN_FILE, index=False, header=False)
df_validate.to_csv(VALIDATE_FILE,index=False,header=False)
df_test.to_csv(TEST_FILE,index=False,header=False)


In [39]:
!head $TRAIN_FILE

0.37236677624,0.954418888228,0.776835996054,0.851796384633,0.250896816986,0.761361604947,0.253067087314,0.265761560233,0.788578744299,0.44815103725,0.498342781707,0.252621701893,0.699964629416,0.43632310053,0.663831963966,0.427084122641,0.453926049357,0.733165466022,0.640814691331,0.619575046384,0.579280053146,0.561688621457,0.50544041774,0.666979214604,0.440562184407,0.542864842631,0.57320066152,0.413947678267,0.312444517277,0.000260400853835,0.0
0.384462243622,0.951870256039,0.777517922889,0.860611459418,0.24570171728,0.766344055428,0.256659915374,0.269241166268,0.785534608599,0.454797071795,0.502797773393,0.226356769684,0.67895247874,0.39120777591,0.655557360131,0.409468121439,0.461562196229,0.715863856981,0.647463086073,0.579508408005,0.58088628888,0.557117879374,0.474739353177,0.664348354025,0.361026569663,0.569120964309,0.442599355817,0.421292008933,0.315439214058,0.000116382444389,0.0
0.245509051345,0.97872170313,0.764600670342,0.855698466285,0.264454827078,0.760344641036,0.2665